In [1]:
import pandas as pd
import fasttext
import numpy as np
from sklearn.metrics import pairwise

import matplotlib.pyplot as plt

In [2]:
plt.style.use("dark_background")

In [3]:
%cd ..

/home/kuba/Projects/github_search


In [4]:
python_files_df = pd.read_csv('data/scraped_python_files.csv')

In [5]:
python_files_df['content'].dropna().to_csv(fasttext_corpus_path, index=False, header=False)

NameError: name 'fasttext_corpus_path' is not defined

In [ ]:
!ls output/python_fasttext.bin

In [6]:
%%time
fasttext_model = fasttext.load_model("output/python_fasttext.bin")

CPU times: user 7.9 s, sys: 2.13 s, total: 10 s
Wall time: 10 s


In [7]:
import ast
paperswithcode_with_imports_df = pd.read_csv('output/papers_with_imports.csv')
paperswithcode_with_imports_df['tasks'] = paperswithcode_with_imports_df['tasks'].str.lower().str.replace("2d ", "").str.replace("3d ", "").str.replace("4d ", "").str.replace("6d ", "").apply(ast.literal_eval)
tasks = paperswithcode_with_imports_df['tasks'].explode().drop_duplicates()

In [8]:
from mlutil.feature_extraction import embeddings

In [9]:
fasttext_embedder = embeddings.FastTextVectorizer(fasttext_model)

In [10]:
fasttext_task_vectors = fasttext_embedder.transform(tasks.values)#np.array(tasks.apply(model.get_sentence_vector).to_list())

In [11]:
def get_text_similarities(embedder, texts, metric=pairwise.cosine_similarity):
    text_vectors = embedder.transform(texts)
    text_similarities = pd.DataFrame(
        data=metric(text_vectors, text_vectors),
        columns=texts,
        index=texts)
    return text_similarities

In [12]:
fasttext_task_similarities = get_text_similarities(fasttext_embedder, tasks)

In [ ]:
word2vec_embedder = embeddings.AverageWordEmbeddingsVectorizer.from_gensim_embedding_model("glove-wiki-gigaword-100")

In [ ]:
fasttext_task_similarities = get_text_similarities(fasttext_embedder, tasks)
word2vec_task_similarities = get_text_similarities(word2vec_embedder, tasks)

In [ ]:
fasttext_task_similarities = pd.DataFrame(
    data=fasttext_task_vectors @ fasttext_task_vectors.T,
    columns=tasks,
    index=tasks)

In [ ]:
import gensim

In [ ]:
%%time
fasttext_gensim_model = gensim.models.FastText.load_fasttext_format("output/python_fasttext.bin")

In [ ]:
task_similarities.loc[''].sort_values()

In [ ]:
fasttext_task_similarities.loc['metric learning'].plot.hist(bins=50)

In [ ]:
task_similarities.loc['few-shot learning'].plot.hist(bins=50)

In [ ]:
fasttext_task_similarities.loc['few-shot learning'].sort_values(ascending=False)[:25]

In [ ]:
word2vec_task_similarities.loc['few-shot learning'].sort_values(ascending=False)[:25]

In [ ]:
fasttext_task_similarities.loc['text generation'].sort_values(ascending=False)[:25]

In [ ]:
word2vec_task_similarities.loc['text generation'].sort_values(ascending=False)[:25]

In [ ]:
fasttext_task_similarities.loc['semantic parsing'].sort_values(ascending=False)[:25]

In [ ]:
word2vec_task_similarities.loc['semantic parsing'].sort_values(ascending=False)[:25]